<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [VAE1] - Variational AutoEncoder (VAE) with MNIST
<!-- DESC --> Episode 1 : Model construction and Training

<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Understanding and implementing a **variational autoencoder** neurals network (VAE)
 - Understanding a more **advanced programming model**

The calculation needs being important, it is preferable to use a very simple dataset such as MNIST to start with.

## What we're going to do :

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard

----
## Bug Note :
**Works in tf 2.0, but not in 2.2/2.3**  

See :
 - https://github.com/tensorflow/tensorflow/issues/34944  
 - https://github.com/tensorflow/probability/issues/519  

Bypass :
 - Use tf 2.0
 - Add `tf.config.experimental_run_functions_eagerly(True)` before compilation...  
Works fine in versions 2.2, 2.3 but with horrible perf. (7s -> 1'50s)
----

## Step 1 - Init python stuff

In [1]:
import numpy as np
import sys, importlib

sys.path.append('..')
import fidle.pwk as pwk

from modules.vae          import VariationalAutoencoder
from modules.loader_MNIST import Loader_MNIST

datasets_dirs = pwk.init('VAE1')

VariationalAutoencoder.about()

**FIDLE 2020 - Practical Work Module**

Version              : 0.6.1 DEV
Notebook id          : VAE1
Run time             : Saturday 19 December 2020, 17:56:34
TensorFlow version   : 2.0.0
Keras version        : 2.2.4-tf
Datasets dir         : /home/pjluc/datasets/fidle
Running mode         : full
Update keras cache   : False
Save figs            : True
Path figs            : ./run/figs


<br>**FIDLE 2020 - Variational AutoEncoder (VAE)**

TensorFlow version   : 2.0.0
VAE version          : 1.28


## Step 2 - Get data

In [2]:
(x_train, y_train), (x_test, y_test) = Loader_MNIST.load()

Dataset loaded.
Normalized.
Reshaped to (60000, 28, 28, 1)


## Step 3 - Get VAE model
Nous allons instancier notre modèle VAE.  
Ce dernier est défini avec une classe python pour alléger notre code.  
La description de nos deux réseaux est donnée en paramètre.  
Notre modèle sera sauvegardé dans le dossier : ./run/<tag>

In [3]:
tag = 'MNIST.001'

input_shape = (28,28,1)
z_dim       = 2
verbose     = 1

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'}
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Conv2DTranspose', 'filters':1,  'kernel_size':(3,3), 'strides':1, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = VariationalAutoencoder(input_shape    = input_shape, 
                             encoder_layers = encoder, 
                             decoder_layers = decoder,
                             z_dim          = z_dim, 
                             verbose        = verbose,
                             run_tag        = tag)
vae.save(model=None)

<br>**Model initialized.**

Outputs will be in  : ./run/MNIST.001


<br>**Encoder :**

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 14, 14, 64)   18496       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 7, 7, 64)     36928       conv2d_1[0][0]                   
____________________________________________________________________________________________

<br>**Decoder :**

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 14, 14, 64)        36928     
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_transpose_3 (Conv2DTr (None, 28, 28, 1)         289 

## Step 4 - Compile it

In [4]:
r_loss_factor = 1000

vae.compile( optimizer='adam', r_loss_factor=r_loss_factor)

Compiled.


## Step 5 - Train
Durations :
 - IDRIS on Jean Zay (V100) : 391.89 sec. - 0:06:31

In [5]:
batch_size        = 100
epochs            = 100
initial_epoch     = 0
k_size            = .1      # 1 mean using 100% of the dataset

In [6]:
vae.train(x_train,
          x_test,
          batch_size        = batch_size, 
          epochs            = epochs,
          initial_epoch     = initial_epoch,
          k_size            = k_size
         )

Train on 6000 samples, validate on 1000 samples
Epoch 1/100
6000/6000 [==============================] - 6s 988us/sample - loss: 104.4901 - vae_r_loss: 101.9539 - vae_kl_loss: 2.5361 - val_loss: 67.7148 - val_vae_r_loss: 66.5085 - val_vae_kl_loss: 1.2063
Epoch 2/100
6000/6000 [==============================] - 5s 841us/sample - loss: 69.0901 - vae_r_loss: 68.0103 - vae_kl_loss: 1.0798 - val_loss: 64.4509 - val_vae_r_loss: 62.6715 - val_vae_kl_loss: 1.7794
Epoch 3/100
6000/6000 [==============================] - 5s 841us/sample - loss: 64.8233 - vae_r_loss: 63.1682 - vae_kl_loss: 1.6551 - val_loss: 59.2293 - val_vae_r_loss: 56.8333 - val_vae_kl_loss: 2.3960
Epoch 4/100
6000/6000 [==============================] - 5s 838us/sample - loss: 59.9215 - vae_r_loss: 57.2821 - vae_kl_loss: 2.6394 - val_loss: 56.9929 - val_vae_r_loss: 54.2415 - val_vae_kl_loss: 2.7514
Epoch 5/100
6000/6000 [==============================] - 5s 889us/sample - loss: 58.3423 - vae_r_loss: 55.4671 - vae_kl_loss: 2.87

KeyboardInterrupt: 

In [7]:
pwk.end()

End time is : Saturday 19 December 2020, 17:58:18
Duration is : 00:01:44 880ms
This notebook ends here


---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>